In [8]:
import cv2
import numpy as np

src = cv2.imread("test.jpg")
src = cv2.resize(src, None, fx=0.5, fy=0.5)
temp = cv2.imread('earphone.jpg')
temp = cv2.resize(temp, None, fx=0.2, fy=0.2)

# mask = np.zeros(src.shape[:2], np.uint8)        # 建立遮罩, 大小和src相同
# # mask[10:220, 10:220] = 3                        # 定義可能前景區域
# # mask[15:160, 15:160] = 1                        # 定義確定前景區域
# bgdModel = np.zeros((1, 65), np.float64)        # 建立內部用暫時計算陣列
# fgdModel = np.zeros((1, 65), np.float64)        # 建立內部用暫時計算陣列
# rect = (15, 20, 180, 200)                       # 建立ROI區域
# src_rect = src.copy()
# # cv2.rectangle(src_rect, rect[0:2], rect[2:4], (0, 0, 255), 2)
# # cv2.imshow('Rectangle', src_rect)

# # 呼叫grabCut()進行分割, 迭代 3 次,回傳mask1
# # 其實mask1 = mask, 因為mask也會同步更新
# mask1, bgd, fgd = cv2.grabCut(src, mask, rect, bgdModel, fgdModel, 3,
#                               cv2.GC_INIT_WITH_RECT)

# # 將 0, 2設為0 --- 1, 3設為1
# mask2 = np.where((mask1 == 0) | (mask1 == 2), 0, 1).astype('uint8')

# dst = src * mask2[:, :, np.newaxis]                 # 計算輸出影像
# cv2.imshow('dst', dst)
h, w = temp.shape[:2]
result = cv2.matchTemplate(src, temp, cv2.TM_CCOEFF_NORMED)
for row in range(len(result)):                                      # 找尋row
    for col in range(len(result[row])):                             # 找尋column
        if result[row][col] > 0.8:                                 # 值大於0.95就算找到了
            dst = cv2.rectangle(
                src, (col, row), (col+w, row+h), (0, 255, 0), 3)
          
cv2.imshow("Dst", dst)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2

tracker_list = []
for i in range(3):
    tracker = cv2.TrackerCSRT_create()        # 創建三組追蹤器
    tracker_list.append(tracker)
colors = [(0,0,255),(0,255,255),(255,255,0)]  # 設定三個外框顏色
tracking = False                              # 設定 False 表示尚未開始追蹤

cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)            # 讀取某個影片
a = 0                                         # 刪減影片影格使用
if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Cannot receive frame")
        break
    frame = cv2.resize(frame,(400,230))       # 縮小尺寸，加快速度
    keyName = cv2.waitKey(1)
    # 為了避免影片影格太多，所以採用 10 格取一格，加快處理速度
    if a%10 == 0:
        if keyName == ord('q'):
            break
        if tracking == False:
            # 如果尚未開始追蹤，就開始標記追蹤物件的外框
            for i in tracker_list:
                area = cv2.selectROI('oxxostudio', frame, showCrosshair=False, fromCenter=False)
                i.init(frame, area)    # 初始化追蹤器
            tracking = True            # 設定可以開始追蹤
        if tracking:
            for i in range(len(tracker_list)):
                success, point = tracker_list[i].update(frame)   # 追蹤成功後，不斷回傳左上和右下的座標
                if success:
                    p1 = [int(point[0]), int(point[1])]
                    p2 = [int(point[0] + point[2]), int(point[1] + point[3])]
                    cv2.rectangle(frame, p1, p2, colors[i], 3)   # 根據座標，繪製四邊形，框住要追蹤的物件

        cv2.imshow('oxxostudio', frame)
    a = a + 1

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.12.0) D:\bld\libopencv_1756075558946\work\modules\core\src\dxt.cpp:3517: error: (-215:Assertion failed) type == CV_32FC1 || type == CV_32FC2 || type == CV_64FC1 || type == CV_64FC2 in function 'cv::dft'


In [ ]:
'''
# 在 Windows 系統上，嘗試使用 DSHOW
# 參數 1 是攝影機索引，第二個參數是 API 偏好
api_preference = cv2.CAP_DSHOW
capture = cv2.VideoCapture(1, api_preference)
'''

import cv2
import numpy as np

def myMatch(image, tmp, thr=0.95, nms_iou=0.3):
    h, w = tmp.shape[:2]
    result = cv2.matchTemplate(image, tmp, cv2.TM_CCOEFF_NORMED)
    # 取出所有高於門檻的點
    boxes, scores = [], []
    for x in range(result.shape[1]):
        for y in range(result.shape[0]):
            boxes.append([int(x), int(y), int(w), int(h)]) # x, y, width, height
            scores.append(float(result[y, x]))
    # 沒找到就回傳空
    if not boxes:
        return []
    # NMS：只保留不重疊（或低重疊）的高分框
    indices = cv2.dnn.NMSBoxes(
        boxes, scores, score_threshold=thr, nms_threshold=nms_iou)
    final = []
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, bw, bh = boxes[i]
            final.append(((x, y), (x + bw, y + bh)))
    return final

src = cv2.imread("test.jpg", cv2.IMREAD_COLOR)
src = cv2.resize(src, None, fx=0.3, fy=0.3)
temps = [cv2.imread("mouse.jpg", cv2.IMREAD_COLOR),
         cv2.imread("earphone.jpg", cv2.IMREAD_COLOR)]
cv2.imshow("src", src)
match = []
for i in range(len(temps)):
    temps[i] = cv2.resize(temps[i], None, fx=0.2, fy=0.2)
    cv2.imshow(f"temps{i}", temps[i])
for i in range(len(temps)):
    h, w = temps[i].shape[:2]
    result = cv2.matchTemplate(src, temps[i], cv2.TM_CCOEFF_N)
    cv2.imshow(f"temps{i}", result)
    # match += myMatch(src, t, thr=0.7)
# for pt1, pt2 in match:
#     cv2.rectangle(src, pt1, pt2, (0, 255, 0), 2)

cv2.waitKey(0)
cv2.destroyAllWindows()